# IMD1103 - Aprendizado por Reforço

### Professor: Dr. Leonardo Enzo Brito da Silva

### Aluno: João Antonio Costa Paiva Chagas

# Laboratório 3: Iteração de política truncada

## Importações

In [ ]:
# In_estadostala os pacotes necessários:
# - gymnasium[toy-text]: inclui ambientes simples como FrozenLake, Taxi, etc.
# - imageio[ffmpeg]: permite salvar vídeos e GIFs (formato .mp4 ou .gif)
!pip install gymnasium[toy-text] imageio[ffmpeg]

In [ ]:
# Importa as bibliotecas principais
import gymnasium as gym               # Biblioteca de simulações de ambientes para RL
import imageio                        # Usada para salvar a sequência de frames como GIF
from IPython.display import Image     # Para exibir a imagem (GIF) diretamente no notebook
import numpy as np                    # Importa o pacote NumPy, amplamente utilizado para manipulação de arrays e operações numéricas
from numpy import linalg as LA        # Rotinas de álgebra linear do NumPy (ex.: normas, autovalores, decomposições)
import matplotlib.pyplot as plt       # Biblioteca para criação de gráficos estáticos em Python (parte do matplotlib)
import seaborn as sns                 # Biblioteca baseada em matplotlib para gráficos estatísticos com visualização mais bonita (usada aqui para heatmaps)
from typing import Dict, Tuple, Optional, List  # Importa ferramentas de tipagem estática do Python
import os

## Armazenamento

In [ ]:
output_dir = "resultados_plots"
os.makedirs(output_dir, exist_ok=True)

## Funções auxiliares para visualização

In [ ]:
def visualizar_politica(
    Pi: np.ndarray,
    env,
    *,
    action_labels: Optional[List[str]] = None,   # default FrozenLake: ["←","↓","→","↑"]
    destacar_gulosa: bool = True,
    suptitle: Optional[str] = "Política (distribuições por estado)",
    save_path: Optional[str] = None
) -> None:
    # Inferir grid do Gymnasium (FrozenLake)
    if not (hasattr(env, "unwrapped") and hasattr(env.unwrapped, "desc")):
        raise ValueError("Passe um ambiente Gymnasium com 'env.unwrapped.desc' (ex.: FrozenLake-v1).")
    desc = env.unwrapped.desc
    desc_str = np.char.decode(desc, "utf-8") if getattr(desc.dtype, "kind", "") == "S" else desc.astype(str)

    n_rows, n_cols = desc_str.shape
    n_estados, n_acoes = Pi.shape
    if n_rows * n_cols != n_estados:
        raise ValueError(f"Incompatibilidade: grid {n_rows}x{n_cols} != n_estados={n_estados}.")

    # Máscaras de terminais
    holes = (desc_str == "H")
    goal  = (desc_str == "G")

    # Rótulos das ações (FrozenLake: LEFT, DOWN, RIGHT, UP)
    if action_labels is None:
        action_labels = ["←", "↓", "→", "↑"]
    if len(action_labels) != n_acoes:
        action_labels = [f"a{i}" for i in range(n_acoes)]

    # Figura
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(3.0 * n_cols, 2.2 * n_rows))
    axs = np.array(axs).reshape(-1)

    for s in range(n_estados):
        r, c = divmod(s, n_cols)
        ax = axs[s]

        # Estados terminais: só fundo colorido, sem barras
        if holes[r, c] or goal[r, c]:
            if holes[r, c]:
                ax.set_facecolor((1.0, 0.0, 0.0, 0.15))  # vermelho translúcido
                ax.set_title(f"Estado {s} (H)")
            else:
                ax.set_facecolor((0.0, 1.0, 0.0, 0.15))  # verde translúcido
                ax.set_title(f"Estado {s} (G)")
            # esconder eixos e ticks
            ax.set_xticks([]); ax.set_yticks([])
            for spine in ax.spines.values():
                spine.set_visible(True)
            continue

        # Estados não-terminais: barras
        pi = Pi[s].astype(float)
        tot = pi.sum()
        if tot > 0:
            pi /= tot  # normalização defensiva

        acoes = np.arange(n_acoes)
        colors = ["gray"] * n_acoes
        if destacar_gulosa:
            colors[int(np.argmax(pi))] = "dimgray"

        ax.bar(acoes, pi, color=colors)
        ax.set_ylim(0, 1.05)
        ax.set_xticks(acoes)
        ax.set_xticklabels(action_labels)
        ax.set_yticks([0, 0.5, 1.0])
        ax.set_title(f"Estado {s}")

    if suptitle:
        fig.suptitle(suptitle, y=1.02, fontsize=12)

    if fig is not None:
        plt.tight_layout()
        if save_path:
            directory = os.path.dirname(save_path)
            if directory:
                os.makedirs(directory, exist_ok=True)
            fig.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close(fig)
            print(f"Plot da política salvo em: {save_path}")
    else:
        plt.show()

def plot_tabular(
    data,
    kind: str = "Q",          # "Q" (valores de ação), "Pi" (política), "V" (valores de estado)
    ambiente=None,            # necessário quando kind="V" para reshape
    ax=None,
    cbar: bool = True,
    fmt: str = ".1f",
    center_zero: bool = True,  # só relevante para "Q" e "V"
    save_path: Optional[str] = None
):
    """
    Plota matrizes tabulares de RL em formato de heatmaps (mapas de calor).
    Esta função cobre 3 casos:
    1. kind="Q": heatmap de Q(s, a) com ações nas linhas e estados nas colunas.
    2. kind="Pi": heatmap de Pi(a|s) (probabilidades) com ações nas linhas e estados nas colunas.
    3. kind="V": heatmap de V(s) no grid (n_rows x n_cols) do ambiente .

    Parameters
    ----------
    data : ndarray
        Dados a serem plotados.
        - Para kind="Q" ou "Pi": array 2D com shape (n_estados, n_acoes).
        - Para kind="V": array 1D com shape (n_estados,) que será remodelado para (ambiente.n_rows, ambiente.n_cols).
    kind : {"Q", "Pi", "V"}, default="Q"
        Tipo do plot:
        - "Q" usa paleta divergente centrada em zero.
        - "Pi" usa paleta sequencial no intervalo [0, 1].
        - "V" plota o valor de estado no grid do ambiente.
    ambiente : object, optional
        Necessário quando kind="V". Deve expor n_rows e n_cols para o reshape.
    ax : matplotlib.axes.Axes, optional
        Eixo onde o heatmap será desenhado. Se None, uma nova figura/eixo é criado.
    cbar : bool, default=True
        Se True, exibe a barra de cores (colorbar).
    fmt : str, default=".1f"
        Formatação dos valores anotados em cada célula do heatmap.
    center_zero : bool, default=True
        Quando kind é "Q" ou "V", centraliza a escala de cores em zero (vmin=-absmax, vmax=absmax). Ignorado para "Pi".

    Returns
    -------
    ax : matplotlib.axes.Axes
        Eixo contendo o heatmap resultante.
    """
    kind = kind.upper()

    xlabel = {"V": "Colunas", "PI": "Estados", "Q": "Estados"}
    ylabel = {"V": "Linhas", "PI": "Ações", "Q": "Ações" }
    title  = {"V": "Valores de Estado (V(s))", "PI": r"Política ($\pi(a|s)$ transposta)", "Q": "Valores de ação (Q(s, a) transposta)"}

    fig = None

    #  V(s): precisa do shape do grid
    match kind:
        case "V":

            if ambiente is None:
                raise ValueError("Para kind='V', passe 'ambiente' para reshape (n_rows, n_cols).")

            if hasattr(ambiente, "n_rows") and hasattr(ambiente, "n_cols"):
                n_rows, n_cols = ambiente.n_rows, ambiente.n_cols
            elif hasattr(ambiente, "unwrapped") and hasattr(ambiente.unwrapped, "desc"):
                # ex.: FrozenLake-v1 (Gymnasium)
                n_rows, n_cols = ambiente.unwrapped.desc.shape
            else:
                raise ValueError(
                    "Passe um objeto com n_rows/n_cols ou um env Gymnasium com .unwrapped.desc."
                )

            M = data.reshape(n_rows, n_cols)

            if ax is None:
                fig, ax = plt.subplots(figsize=(n_cols, n_rows))

            if center_zero:
                vmax = float(np.abs(M).max())
                vmin = -vmax
            else:
                vmin = float(M.min())
                vmax = float(M.max())

            cmap, square = "bwr", True

        case "PI" | "Q":

            # Q(s,a) e Pi(a|s): ações nas linhas, estados nas colunas
            M = data.T  # data: (n_estados, n_acoes) -> transposto para (n_acoes, n_estados)
            n_acoes, n_estados = M.shape

            if ax is None:
                fig, ax = plt.subplots(figsize=(n_estados, n_acoes))

            if kind == "PI":
                cmap = "Blues";
                vmin, vmax = 0.0, 1.0
            else:  # "Q"
                cmap = "bwr"
                if center_zero:
                    vmax = float(np.abs(M).max())
                    vmin = -vmax
                else:
                    vmin = float(M.min())
                    vmax = float(M.max())

            square = False

        case _:
            raise ValueError(f"kind desconhecido: {kind!r} (use 'Q', 'Pi' ou 'V').")


    ax = sns.heatmap(
        data=M,
        annot=True,
        fmt=fmt,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        cbar=cbar,
        square=square,
        linewidths=0.5,
        linecolor="gray",
        ax=ax
    )

    ax.set_xlabel(xlabel[kind])
    ax.set_ylabel(ylabel[kind])
    ax.set_title(title[kind])

    # bordas externas
    for side in ("left", "right", "top", "bottom"):
        ax.spines[side].set_visible(True)
        ax.spines[side].set_linewidth(0.5)
        ax.spines[side].set_edgecolor("gray")

    # rótulos
    if kind in ("Q", "PI"):
        ax.set_xticks(np.arange(n_estados) + 0.5)
        ax.set_xticklabels([f"s{i}" for i in range(n_estados)], rotation=0)
        ax.set_yticks(np.arange(n_acoes) + 0.5)
        ax.set_yticklabels([f"a{i}" for i in range(n_acoes)], rotation=0)

    if fig is not None:
      plt.tight_layout()
      if save_path:
          directory = os.path.dirname(save_path)
          if directory:
              os.makedirs(directory, exist_ok=True)
          fig.savefig(save_path, dpi=300, bbox_inches='tight')
          plt.close(fig)
          print(f"Plot tabular '{kind}' salvo em: {save_path}")
      else:
          plt.show()

    return

In [ ]:
def plotar_grafico_dispersao(
    j_truncado_lista: List[int],
    iteracoes_k: List[int],
    map_name: str,
    is_slippery: bool,
    save_path: Optional[str] = None
) -> None:
    """
    Gera um gráfico de dispersão mostrando o número de iterações (k) para
    convergir em função do número de varreduras de avaliação (j_truncado).

    Parâmetros
    ----------
    j_truncado_lista : List[int]
        Valores de j_truncado testados (eixo x).
    iteracoes_k : List[int]
        Número de iterações k correspondente a cada j_truncado (eixo y).
    map_name : str
        Nome do mapa do ambiente (usado no título).
    is_slippery : bool
        Se o ambiente é estocástico (usado no título).
    """
    plt.figure(figsize=(10, 6))
    plt.scatter(j_truncado_lista, iteracoes_k, marker='o', s=80, alpha=0.8, label='Dados do Experimento')
    plt.plot(j_truncado_lista, iteracoes_k, linestyle='--', alpha=0.5, label='Linha de Tendência')

    plt.title(f'Convergência do Algoritmo vs. j_truncado (Mapa: {map_name}, Slippery: {is_slippery})', fontsize=14)
    plt.xlabel('j_truncado (Varreduras de Avaliação por Iteração)', fontsize=12)
    plt.ylabel('Iterações Externas para Convergência (k)', fontsize=12)
    plt.xticks(j_truncado_lista)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)

    if save_path:
        plt.tight_layout()
        directory = os.path.dirname(save_path)
        if directory:
            os.makedirs(directory, exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Gráfico de dispersão salvo em: {save_path}")
    else:
        plt.show()

## Algoritmo: Iteração de política truncada

### Avaliar política truncada

In [ ]:
def avaliar_politica_truncada(
    env,
    Pi: np.ndarray,
    j_truncado: int,
    V: np.ndarray | None = None,
    gamma: float = 0.9,
) -> np.ndarray:
    """
    Avaliação de política truncada.

    Executa exatamente j_truncado varreduras do algoritmo de avaliação de política (Jacobi).

    Atualização:
        V_{new}(s) = sum_a Pi[a|s] * sum_{(p,s',r,done) in env.P[s][a]} p * [ r + gamma * V_old[s'] ]

    Parâmetros
    ----------
    env : gym.Env (unwrapped)
        Ambiente com dicionário de transições env.P: Dict[s][a] -> List[(p, s', r, done)].
    Pi : np.ndarray, shape (n_estados, n_acoes)
        Política.
    j_truncado : int
        Número de iterações (varreduras de avaliação).
    V : np.ndarray | None, shape (n_estados,), opcional
        Valores de estado iniciais. Se None, começa em zeros.
    gamma : float, default=0.9
        Fator de desconto.

    Retorna
    -------
    V : np.ndarray, shape (n_estados,)
        Valores de estado após j_truncado varreduras.
    """

    n_estados = env.observation_space.n
    n_acoes   = env.action_space.n

    # Inicializações
    if V is None:
        V = np.zeros(n_estados, dtype=float)

    ############################################################################################################
    # AVALIAÇÃO DA POLÍTICA ATUAL
    ############################################################################################################
    # Código aqui

    # V_atual representa o vetor de valores V que será atualizado a cada iteração interna
    # Começa como uma cópia do V da iteração de política anterior
    V_atual = V.copy()

    # Executa um número fixo de varreduras (j_truncado) para avaliar a política
    for _ in range(j_truncado):
        # V_proximo armazenará os novos valores calculados nesta varredura.
        V_proximo = np.zeros(n_estados, dtype=float)

        # Itera sobre cada estado para calcular seu novo valor.
        for estado in range(n_estados):
            valor_estado = 0

            # Itera sobre cada ação possível a partir do estado.
            # Pi[estado] é a distribuição de probabilidade das ações para o estado atual.
            for acao, prob_acao in enumerate(Pi[estado]):

                # Calcula o valor apenas para ações com probabilidade > 0.
                if prob_acao > 0:
                    valor_acao_esperado = 0

                    # Calcula o valor esperado de tomar a ação no estado.
                    for prob, proximo_estado, recompensa, _ in env.P[estado][acao]:
                        valor_acao_esperado += prob * (recompensa + gamma * V_atual[proximo_estado])

                    # Pondera o valor esperado da ação pela probabilidade de escolhê-la.
                    valor_estado += prob_acao * valor_acao_esperado

            # Atribui o valor calculado para o estado no vetor da próxima iteração.
            V_proximo[estado] = valor_estado

        # Atualiza V_atual com os valores recém-calculados para a próxima varredura.
        V_atual = V_proximo.copy()

    ############################################################################################################

    return V_atual


### Melhorar política truncada

In [ ]:
def melhorar_politica(
    env,
    V: np.ndarray,
    gamma: float = 0.9,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Melhoria de política.

    Dado V, calcula:
        Q(s,a) = sum_{(p,s',r,done) in P[s][a]} p * (r + gamma * V[s'])
    e retorna a política gulosa determinística.

    Parâmetros
    ----------
    env : gym.Env (unwrapped)
        Ambiente com dicionário de transições env.P: Dict[s][a] -> List[(p, s', r, done)].
    V : np.ndarray, shape (n_estados,)
        Valores de estado.
    gamma : float, default=0.99
        Fator de desconto (0 <= gamma < 1).

    Retorna
    -------
    Q : np.ndarray, shape (n_estados, n_acoes)
        Valores de ação.
    Pi_nova : np.ndarray, shape (n_estados, n_acoes)
        Política gulosa determinística.
    """
    # Dimensões e validações

    n_estados = env.observation_space.n
    n_acoes   = env.action_space.n

    # Inicializações
    Q       = np.zeros((n_estados, n_acoes), dtype=float)
    Pi_nova = np.zeros((n_estados, n_acoes), dtype=float)

    ############################################################################################################
    # MELHORIA DA POLÍTICA
    ############################################################################################################
    # Código aqui

    # Itera sobre cada estado para calcular os valores de Q e determinar a melhor ação
    for estado in range(n_estados):
        # Calcula o valor Q para cada ação possível no estado atual
        for acao in range(n_acoes):
            valor_acao_esperado = 0

            # Soma sobre todos os possíveis resultados (s', r) para o par (estado, acao)
            for prob, proximo_estado, recompensa, _ in env.P[estado][acao]:
                valor_acao_esperado  += prob * (recompensa + gamma * V[proximo_estado])

            Q[estado, acao] = valor_acao_esperado

        # Após calcular Q(estado, acao) para todas as ações em determinado estado, encontra a melhor ação (aquela com o maior valor Q)
        melhor_acao = np.argmax(Q[estado])

        # Cria uma política determinística que sempre escolhe a melhor ação
        Pi_nova[estado, melhor_acao] = 1.0
    ############################################################################################################

    return Q, Pi_nova

### Algoritmo: iteração de política truncada

In [ ]:
def iteracao_de_politica_truncada(
    env,
    gamma: float = 0.99,
    j_truncado: int = 10,
    theta: float = 1e-8,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, int]:
    """
    Iteração de Política truncada.

    Loop externo (k):
      1) V <- avaliar_politica_truncada(env, Pi_k, j_truncado, V, gamma)
      2) (Q, Pi_{k+1}) <- melhorar_politica(env, V, gamma)
      3) parar quando ||V - V_prev||_inf < theta

    Parâmetros
    ----------
    env : gym.Env (unwrapped)
        Ambiente com dicionário de transições env.P: Dict[s][a] -> List[(p, s', r, done)].
    gamma : float, default=0.9
        Fator de desconto (0 <= gamma < 1).
    j_truncado : int, default=10
        Número de varreduras na avaliação de política truncada.
    theta : float, default=1e-8
        Critério de parada baseado em V entre iterações externas (convergência).

    Retorna
    -------
    V  : np.ndarray, shape (n_estados,)
        Valores de estado.
    Q  : np.ndarray, shape (n_estados, n_acoes)
        Valores de ação da última melhoria.
    Pi : np.ndarray, shape (n_estados, n_acoes)
        Política determinística.
    k  : int
        Número de iterações externas executadas.
    """
    # Dimensões e validações básicas
    n_estados = env.observation_space.n
    n_acoes   = env.action_space.n

    # Inicializações
    V  = np.zeros(n_estados, dtype=float)
    Q  = np.zeros((n_estados, n_acoes), dtype=float)
    Pi = np.full((n_estados, n_acoes), 1.0 / n_acoes, dtype=float)  # política uniforme

    # Laço externo
    k = 0
    while True:

        k += 1

        V_prev = V.copy()

        # 1) Avaliação de política (truncada)
        V = avaliar_politica_truncada(
            env=env,
            Pi=Pi,
            j_truncado=j_truncado,
            V=V,
            gamma=gamma,
        )

        # 2) Melhoria de política (gulosa)
        Q, Pi_nova = melhorar_politica(
            env=env,
            V=V,
            gamma=gamma,
        )
        Pi = Pi_nova

        # 3) Critério de parada baseado apenas em V
        if np.linalg.norm(V - V_prev, ord=np.inf) < theta:
            break

    return V, Q, Pi, k


## Experimento

In [ ]:
def executar_e_plotar_experimento_truncado(
    nome_experimento: str,
    env,
    output_dir: str,
    gamma: float,
    j_truncado: int,
    theta: float
):
    """
    Executa a Iteração de Política Truncada para uma configuração específica,
    imprime a convergência e salva todas as figuras de visualização.
    """
    print(f"\n==============================================")
    print(f"  Executando: {nome_experimento}")
    print(f"==============================================\n")

    # Roda o algoritmo
    V, Q, Pi, k = iteracao_de_politica_truncada(
        env,
        gamma=gamma,
        j_truncado=j_truncado,
        theta=theta
    )

    print(f"--> Convergência em {k} iterações para j_truncado={j_truncado}.\n")

    # Gera e salva todas as figuras
    print("Salvando figuras...")
    plot_tabular(
        V, kind="V", ambiente=env, center_zero=False,
        save_path=f"{output_dir}/{nome_experimento}_V.pdf"
    )
    plot_tabular(
        Q, kind="Q",
        save_path=f"{output_dir}/{nome_experimento}_Q.pdf"
    )
    plot_tabular(
        Pi, kind="Pi",
        save_path=f"{output_dir}/{nome_experimento}_Pi.pdf"
    )
    visualizar_politica(
        Pi, env=env, suptitle=f"Política - {nome_experimento}",
        save_path=f"{output_dir}/{nome_experimento}_Politica.pdf"
    )
    print("Figuras salvas!")

In [ ]:
## Função para Executar os Experimentos

def executar_experimento_convergencia(
    map_name: str,
    is_slippery: bool,
    j_truncado_lista: List[int],
    gamma: float,
    theta: float,
    save_path: str
) -> None:
    """

    Executa o algoritmo de iteração de política truncada para diferentes

    valores de j_truncado e plota um gráfico de dispersão dos resultados.

    """

    env = gym.make("FrozenLake-v1", map_name=map_name, is_slippery=is_slippery)
    env = env.unwrapped
    iteracoes_k = []

    print(f"--- Iniciando experimento para o mapa {map_name} ---")

    for j in j_truncado_lista:
        _, _, _, k = iteracao_de_politica_truncada(env, gamma=gamma, j_truncado=j, theta=theta)
        iteracoes_k.append(k)

        print(f"j_truncado = {j:3d} -> k = {k:3d} iterações para convergir")

    print("--- Experimento finalizado ---")

    # Chama a função de plotagem com os dados reais coletados
    plotar_grafico_dispersao(
        j_truncado_lista=j_truncado_lista,
        iteracoes_k=iteracoes_k,
        map_name=map_name,
        is_slippery=is_slippery,
        save_path=save_path
    )

## Tarefa

1. Implemente o algoritmo **iteração de política truncada**.
2. Gere um **gráfico de dispersão** em que cada ponto (x,y) corresponde à (valor do j_truncado, iteração em que a condição de convergência foi satisfeita para este j_truncado).

** Utilize a seguinte configuração do ambiente FrozenLake para os experimentos**

- `map_name = '8x8'` e `map_name = '4x4'`      
- `render_mode="rgb_array"`
- `is_slippery=True`

**No experimento com configuração `map_name = '4x4'` mostrar:**

1. **Figuras**:
   - heatmap de $V(s)$ (função `plot_tabular`);
   - heatmap de $Q(s,a)$ (função `plot_tabular`);
   - heatmap de $\pi(a\mid s)$ (função `plot_tabular`);
   - gráficos de barras de $\pi(a\mid s)$ (função `visualizar_politica`).
   - gráfico de dispersão

**No experimento com configuração `map_name = '8x8'` mostrar:**

1. **Figura**:
   - gráfico de dispersão

**Entregáveis:**

2. **Código** (notebook `.ipynb`)
1. **Relatório** (`.pdf`).
- O PDF deve conter:
  - **Setup** (parâmetros usados).
  - **Resultados** (figuras e tabelas organizadas por experimento).
  - **Análises curtas** por experimento.
- O PDF **NÃO** deve conter:
    - Códigos.

### 2:

In [ ]:
print("\n################# TAREFA 2 #################")

In [ ]:
GAMMA = 0.95
THETA = 1e-8
IS_SLIPPERY = True
J_TRUNCADO_LISTA = [1, 2, 3, 5, 10, 15, 20, 50, 100]

#### 4x4:

In [ ]:
print("\n################# EXPERIMENTO MAPA 4x4 #################")

env_4x4 = gym.make("FrozenLake-v1", map_name='4x4', is_slippery=IS_SLIPPERY)
env_4x4 = env_4x4.unwrapped

executar_e_plotar_experimento_truncado(
    nome_experimento="4x4_j_truncado_20",
    env=env_4x4,
    output_dir=output_dir,
    gamma=GAMMA,
    j_truncado=20,
    theta=THETA
)

executar_experimento_convergencia(
    map_name='4x4',
    is_slippery=IS_SLIPPERY,
    j_truncado_lista=J_TRUNCADO_LISTA,
    gamma=GAMMA,
    theta=THETA,
    save_path=f"{output_dir}/4x4_convergencia.pdf"
)

#### 8x8:

In [ ]:
print("\n################# EXPERIMENTO MAPA 8x8 #################")

executar_experimento_convergencia(
    map_name='8x8',
    is_slippery=IS_SLIPPERY,
    j_truncado_lista=J_TRUNCADO_LISTA,
    gamma=GAMMA,
    theta=THETA,
    save_path=f"{output_dir}/8x8_convergencia.pdf"
)

In [ ]:
!zip -r /content/resultados_plots.zip {output_dir}